<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 7 Hypothesis Testing II ###

In [ ]:
### standard imports
import numpy as np
import pandas as pd

#import os
from scipy.stats import ttest_ind, f_oneway, chi2_contingency, ks_2samp

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
import seaborn as sns

import random

Here is the documentation of all the tests we have discussed during class and many more...

[`scipy.stats statistical tests`](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests)

## Drug Consumption Data Set ##

This data contains survey records for 1885 respondents. Participants were questioned concerning their use of 18 legal and illegal drugs (alcohol, amphetamines, amyl nitrite, benzodiazepine, cannabis, chocolate, cocaine, caffeine, crack, ecstasy, heroin, ketamine, legal highs, LSD, methadone, mushrooms, nicotine and volatile substance abuse and one fictitious drug (Semeron) which was introduced to identify over-claimers. For each drug use they have to select one of the 7 categories: 

- "Never Used"
- "Used over a Decade Ago"
- "Used in Last Decade"
- "Used in Last Year"
- "Used in Last Month"
- "Used in Last Week"
- "Used in Last Day"

In addition, data also include demographic variables: level of education, age, gender, country of residence ethnicity and personality measurements which include NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness), BIS-11 (impulsivity), and ImpSS (sensation seeking).

All input attributes are originally categorical and are quantified. After quantification values of all input features can be considered as real-valued. 


[Data Source](https://archive.ics.uci.edu/ml/datasets/Drug+consumption+%28quantified%29)

In [ ]:
# read in data
drug = pd.read_csv('drug_consumption.csv')
# print number of rows and columns
print(drug.shape)
# print column/variable names
print(drug.columns)
# look at a few rows
drug.head()

In [ ]:
# summary statistics on numerical variables
drug.describe()

In [ ]:
# make some plots to visualize personality score distribution

def plot_score(df, score):
    ''' Given dataframe name and
    column name for the personality score,
    will plot histogram of the score by gender
    '''
    men = df[df['Gender']=='Male'][score]
    women = df[df['Gender']=='Female'][score]
    plt.hist(men, label='men', density=True, alpha=0.4)
    plt.hist(women, label='women', density=True, alpha=0.4)
    plt.xlabel('standardized score')
    plt.title('Personality Score '+ score)
    plt.legend();

In [ ]:
# test the function
# 'Nscore','Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS',
plot_score(df=drug, score='Impulsive')

### Two sample t-test ###


**Question**: Do men and women have the same personality scores on average? 

Let's use a subset of the data to do the test. You change change the sample size later and see how it changes the test results. 

In [ ]:
sample_size = 200
# sample without replacement
drug_test = drug.sample(sample_size, random_state=2023)

In [ ]:
# Student's t-test
men = drug_test[drug_test['Gender']=='Male']['Impulsive']
women = drug_test[drug_test['Gender']=='Female']['Impulsive']
ttest_results = ttest_ind(men, women)
print(ttest_results)

In [ ]:
# visualize
plot_score(df=drug_test, score='Impulsive')

In [ ]:
# Run t-test to compare personality score between gender groups
def t_test_result(score, ref_pvalue=0.05):
    ''' Given column name for the personality score 
    and a reference p-value (0.05 by default),
    will print a string indicating tests results
    '''
    men = drug_test[drug_test['Gender']=='Male'][score]
    women = drug_test[drug_test['Gender']=='Female'][score]
    ttest_results = ttest_ind(men, women)
    p = ttest_results[1]
    if p < ref_pvalue:
        result= score + " is significantly different between male and female with a p-value = %.3f"
    else:
        result= score + " is NOT significantly different between male and female with a p-value = %.3f"

    print(result % p)

In [ ]:
# test the function
t_test_result(score='Impulsive', ref_pvalue=0.05)

In [ ]:
# run test on each personality score with a loop
scores = ['Nscore','Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS']

for score in scores:
    t_test_result(score, ref_pvalue=0.05)

### ANOVA test ###

**Question**: Do people from different countries have the same personality scores on average? 


In [ ]:
# create a boxplot to compare personality score across demographic
# change x variable to other scores
# change y variable to other categorical: Ethnicity, Age, Education etc.
sns.boxplot(x='Impulsive', y='Country', data=drug);

In [ ]:
# look at number of observations in each country
drug['Country'].value_counts()

In [ ]:
# one-way ANOVA to compare 4 countries
UK = drug[drug['Country'] == 'UK']['Impulsive']
USA = drug[drug['Country'] == 'USA']['Impulsive']
Canada = drug[drug['Country'] == 'Canada']['Impulsive']
Australia = drug[drug['Country'] == 'Australia']['Impulsive']

# input multiple arrays for anova test
anova_results = f_oneway(UK, USA, Canada, Australia)
print(anova_results)

In [ ]:
# post hoc paiewise t-test
print(ttest_ind(UK, USA))
print(ttest_ind(UK, Canada))
print(ttest_ind(UK, Australia))
print(ttest_ind(USA, Canada))
print(ttest_ind(USA, Australia))
print(ttest_ind(Canada, Australia))

### Chi-square test of Independence###

**Question**: Is education level and alcohol consumption independent? 

In [ ]:
# remember how to create a pivot table?
# fill with 0 so there is no NaN
contingency_table = drug.pivot_table(index='Education', columns='Alcohol', 
                                     values='ID', aggfunc='count', fill_value=0)
contingency_table

In [ ]:
# input is a contingency table
# output is test stat, p-value, df, chi2 table
chisquare_results = chi2_contingency(contingency_table)
print(chisquare_results)
# print just p-values
#print('Chisquare test p-value:', chisquare_results[1])

### Nonparametric two sample Kolmogorov–Smirnov test ###

**Question**: Do men and women have the same personality scores distribution? 

**No normality assumption!**

In [ ]:
men = drug_test[drug_test['Gender']=='Male']['Nscore']
women = drug_test[drug_test['Gender']=='Female']['Nscore']
ks_2samp_results = ks_2samp(men, women)
print(ks_2samp_results)

In [ ]:
men = drug_test[drug_test['Gender']=='Male']['SS']
women = drug_test[drug_test['Gender']=='Female']['SS']
ks_2samp_results = ks_2samp(men, women)
print(ks_2samp_results)